In [ ]:
import numpy as np
import obi_one as obi
import os
import pandas as pd
from conntility.connectivity import ConnectivityMatrix

In [ ]:
input_root = "/Users/pokorny/Data/Circuits"
cmat_root = "/Users/pokorny/Data/ConnectivityMatrices"
output_root = "../../../obi-output/extracted_small_microcircuits_volumetric"

In [ ]:
# Compute population centroid
circuit = obi.Circuit(name="nbS1-O1", path=os.path.join(input_root, "nbS1-O1", "circuit_config_postfix2.json"))
c = circuit.sonata_circuit
nodes = c.nodes[circuit.default_population_name]
population_centroid = nodes.positions().mean()

In [ ]:
# nbS1-O1 - Central neurons in connectivity
df_sel = pd.read_pickle("Top10CentralNeurons_Layer_Hex.pkl")  # (Contains neuron IDs relative to nbS1-HEXn)
df_opt = pd.read_pickle("Top10CentralNeurons_Layer_Hex_OptIDX.pkl")

# Load x/y/z positions from connectivity matrix and select best neuron in each layer
base_name = "nbS1"
hex_sel = "hex0"
layers = range(1, 7)
cmat = ConnectivityMatrix.from_h5(os.path.join(cmat_root, f"{base_name}-{hex_sel.upper()}", "connectivity_matrix.h5"))
ntab = cmat.vertices.set_index("node_ids")

xyz_selection = []
for _lay in layers:
    opt_idx = df_opt.loc[(hex_sel, _lay)]["pos"]
    node_id = df_sel[("hex0", _lay)].iloc[opt_idx].tolist()
    xyz_selection.append(ntab.loc[node_id][["x", "y", "z"]].astype(float).to_list())
    assert ntab.loc[node_id]["layer"] == str(_lay), "ERROR: Layer mismatch!"
xyz_selection = pd.DataFrame(xyz_selection, columns=["x", "y", "z"], index=layers) - population_centroid  # Relative to population centroid
xyz_selection.index.name = "layer"

In [ ]:
# nbS1-O1 - Central neurons in space
hex_sel = "hex0"
layers = range(1, 7)

# Compute layer centers
pos_tab = nodes.get("hex0", properties=["layer", "x", "y", "z"])
layer_centers = pos_tab.groupby("layer").mean()
layer_centers.index = layer_centers.index.astype(int)
layer_centers = layer_centers.loc[layers]

# Find position of closest neurons to layer centers
xyz_selection_space = []
for _lay in layers:
    sel_idx = np.argmin(np.sum((pos_tab[["x", "y", "z"]] - layer_centers.loc[_lay])**2, 1))
    xyz_selection_space.append(pos_tab.iloc[sel_idx][["x", "y", "z"]].astype(float).to_list())
xyz_selection_space = pd.DataFrame(xyz_selection_space, columns=["x", "y", "z"], index=layers) - population_centroid  # Relative to population centroid
xyz_selection_space.index.name = "layer"

In [ ]:
# Set up volumetric neuron sets

# # (1) Centered around a central neurons in the network's connectivity
# N = 10
# volumetric_neuron_sets = obi.VolumetricCountNeuronSet(ox=xyz_selection["x"].to_list(), oy=xyz_selection["y"].to_list(), oz=xyz_selection["z"].to_list(), n=N,
#                                                       property_filter=obi.NeuronPropertyFilter(filter_dict={}))
# virtual_sources_to_ignore = [("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", "VPM") if _lay == 1 else ("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", ) for _lay in layers]
# output_path = os.path.join(output_root, "nCN")

# (2) Centered around a central neurons in space
N = 10
volumetric_neuron_sets = obi.VolumetricCountNeuronSet(ox=xyz_selection_space["x"].to_list(), oy=xyz_selection_space["y"].to_list(), oz=xyz_selection_space["z"].to_list(), n=N,
                                                      property_filter=obi.NeuronPropertyFilter(filter_dict={}))
virtual_sources_to_ignore = [("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", "VPM") if _lay == 1 else ("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", ) for _lay in layers]
output_path = os.path.join(output_root, "sCN")

In [ ]:
# # nbS1-O1 - PLOTTING (Central in connectivity)
# import matplotlib.pyplot as plt

# # cmat_O1 = ConnectivityMatrix.from_h5(os.path.join(cmat_root, f"{base_name}-O1", "connectivity_matrix.h5"))

# # Load x/y/z positions from connectivity matrix
# base_name = "nbS1"
# plt.figure()
# for hex_sel in [f"hex{_i}" for _i in range(7)]:
#     cmat = ConnectivityMatrix.from_h5(os.path.join(cmat_root, f"{base_name}-{hex_sel.upper()}", "connectivity_matrix.h5"))
#     plt.plot(cmat.vertices["y"], cmat.vertices["z"], '.')
# # plt.plot(cmat_O1.vertices["y"], cmat_O1.vertices["z"], ',k')
# # plt.plot(xyz_selection["y"], xyz_selection["z"], "xk")
# plt.show()

# # df_sel[("hex0", 1)]

In [ ]:
# # PLOTTING (Central in space)
# import matplotlib.pyplot as plt

# plt.figure()
# for _lay in layers:
#     plt.plot(pos_tab[pos_tab["layer"] == str(_lay)]["x"] - population_centroid.loc["x"], pos_tab[pos_tab["layer"] == str(_lay)]["y"] - population_centroid.loc["y"], ".")
# for _lay in layers:
#     plt.plot(xyz_selection_space.loc[_lay]["x"], xyz_selection_space.loc[_lay]["y"], "*k")
# plt.show()

In [ ]:
circuit_extractions_scan_config = obi.CircuitExtractionScanConfig(
                    initialize=obi.CircuitExtractionScanConfig.Initialize(
                        circuit=circuit,
                        run_validation=False,
                        do_virtual=True,
                        create_external=True,
                        virtual_sources_to_ignore=virtual_sources_to_ignore,
                    ),
                    # neuron_set=obi.PredefinedNeuronSet(node_set=["Excitatory", "Inhibitory"], random_sample=[100, 500]),
                    # neuron_set=obi.PredefinedNeuronSet(node_set=["Layer2", "Layer3", "Layer4", "Layer5", "Layer6"], random_sample=10),
                    # neuron_set=obi.IDNeuronSet(neuron_ids=node_id_lists),
                    neuron_set=volumetric_neuron_sets
)

coupled_scan = obi.CoupledScanGenerationTask(form=circuit_extractions_scan_config, output_root=output_path, coordinate_directory_option="ZERO_INDEX")

In [ ]:
# Display the intermediary data
coupled_scan.multiple_value_parameters(display=True)
coupled_scan.coordinate_parameters(display=True)
# coupled_scan.coordinate_instances(display=True)

In [ ]:
coupled_scan.execute()
obi.run_tasks_for_generated_scan(coupled_scan)

In [ ]:
# Check circuits
from bluepysnap import Circuit

for _idx, _inst in enumerate(coupled_scan.coordinate_instances()):
    cfg = coupled_scan.output_root / str(_idx) / "circuit_config.json"
    c = Circuit(cfg)
    nodes = c.nodes['S1nonbarrel_neurons']
    mtypes = np.unique(nodes.get(properties=['mtype']), return_counts=True)
    mt_hist = [f"{_c}x{_mt}" for _mt, _c in zip(*mtypes)]
    print(f"L{layers[_idx]}: {nodes.size} neurons, {c.edges["S1nonbarrel_neurons__S1nonbarrel_neurons__chemical"].size} synapses, m-types: {mt_hist}")
